In [3]:
%pylab inline
import sqlite3
import pandas
import tqdm
from sklearn import *
import ml_metrics
from xgboost import sklearn as xgblearn

Populating the interactive namespace from numpy and matplotlib


/home/riri/.virtualenvs/mxnet-3/lib/python3.4/site-packages/sklearn/lda.py:4: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarning)
/home/riri/.virtualenvs/mxnet-3/lib/python3.4/site-packages/sklearn/qda.py:4: DeprecationWarning: qda.QDA has been moved to discriminant_analysis.QuadraticDiscriminantAnalysis in 0.17 and will be removed in 0.19.
  "in 0.17 and will be removed in 0.19.", DeprecationWarning)


In [10]:
admissible_cols = ['sales_depo', 'sales_channel', 'route_id', 'client_id', 'product_id']

def fit_mini_batch(reg, test=False, batch=None):
    reg.n_estimators += 4
    if batch == None:
        batch = np.random.randint(50)
    con = sqlite3.connect('/tmp/train_test_data.sqlite3')
    try:
        print('Fetching data: %d' % batch)
        data = pandas.read_sql('''
            SELECT * 
              FROM train_data 
             WHERE adjusted_demand is not null 
                   AND week_num < 8
                   AND rand in (?, ?)''', con=con, params=[batch, batch + 50])
    finally:
        con.close()
    reg.fit(data[admissible_cols].as_matrix().copy(order='C'), 
            data.adjusted_demand.as_matrix().copy(order='C'))

    if test:
        con = sqlite3.connect('/tmp/train_test_data.sqlite3')
        try:
            data = pandas.read_sql('''
                SELECT * 
                  FROM test_data 
                 WHERE adjusted_demand is not null 
                       AND week_num >= 8
                       AND rand = ?''', con=con, params=[batch])
        finally:
            con.close()
        pred = reg.predict(data[admissible_cols].as_matrix().copy(order='C'))
        print(ml_metrics.rmsle(data.adjusted_demand, pred))

In [13]:
reg = ensemble.GradientBoostingRegressor(
    warm_start=True,
    max_depth=8,
    verbose=0, 
    n_estimators=0)

In [14]:
for i in range(100):
    fit_mini_batch(reg, True)

Fetching data: 11


ValueError: Found array with 0 sample(s) (shape=(0, 5)) while a minimum of 1 is required.